In [ ]:
##### Beginning of file

# This file was generated by PredictMD version 0.20.0-DEV
# For help, please visit https://www.predictmd.net

import PredictMD

### Begin project-specific settings

PredictMD.require_julia_version("v0.7.0")

PredictMD.require_predictmd_version("0.20.0-DEV")

# PredictMD.require_predictmd_version("0.20.0-DEV", "0.21.0-")

PROJECT_OUTPUT_DIRECTORY = PredictMD.project_directory(
    homedir(),
    "Desktop",
    "breast_cancer_biopsy_example",
    )

### End project-specific settings

### Begin C-SVC code

display(c_svc_svm_classifier_hist_testing)

PredictMD.singlelabelbinaryclassificationmetrics(
    c_svc_svm_classifier,
    smoted_training_features_df,
    smoted_training_labels_df,
    single_label_name,
    positive_class;
    sensitivity = 0.95,
    )

PredictMD.singlelabelbinaryclassificationmetrics(
    c_svc_svm_classifier,
    testing_features_df,
    testing_labels_df,
    single_label_name,
    positive_class;
    sensitivity = 0.95,
    )

c_svc_svm_classifier_filename = joinpath(
    PROJECT_OUTPUT_DIRECTORY,
    "c_svc_svm_classifier.jld2",
    )

PredictMD.save_model(c_svc_svm_classifier_filename, c_svc_svm_classifier)

### End C-SVC code

##### End of file

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*